<a href="https://colab.research.google.com/github/hmatny/question_answering/blob/master/Anserini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Setup

#### Download and install maven

In [0]:
!wget http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
!tar -xvzf apache-maven-3.6.1-bin.tar.gz -C /opt

--2019-04-19 20:07:24--  http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
Resolving mirror.reverse.net (mirror.reverse.net)... 208.100.14.200
Connecting to mirror.reverse.net (mirror.reverse.net)|208.100.14.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9136463 (8.7M) [application/x-gzip]
Saving to: ‘apache-maven-3.6.1-bin.tar.gz’

apache-maven-3.6.1- 100%[===================>]   8.71M  2.67MB/s    in 3.3s    

2019-04-19 20:07:27 (2.67 MB/s) - ‘apache-maven-3.6.1-bin.tar.gz’ saved [9136463/9136463]

apache-maven-3.6.1/README.txt
apache-maven-3.6.1/LICENSE
apache-maven-3.6.1/NOTICE
apache-maven-3.6.1/lib/
apache-maven-3.6.1/lib/slf4j-api.license
apache-maven-3.6.1/lib/checker-compat-qual.license
apache-maven-3.6.1/lib/jsr250-api.license
apache-maven-3.6.1/lib/jcl-over-slf4j.license
apache-maven-3.6.1/lib/org.eclipse.sisu.plexus.license
apache-maven-3.6.1/lib/animal-sniffer-annotations.license
apache-maven-3.6.

#### Set environment variables

In [0]:
import os
os.environ['PATH'] += ':/usr/lib/jvm/java-8-openjdk-amd64/bin:/opt/apache-maven-3.6.1/bin'
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['M2_HOME'] = "/opt/apache-maven-3.6.0"

# sanity check
!mvn -version

Apache Maven 3.6.1 (d66c9c0b3152b2e69ee9bac180bb8fcc8e6af555; 2019-04-04T19:00:29Z)
Maven home: /opt/apache-maven-3.6.1
Java version: 1.8.0_191, vendor: Oracle Corporation, runtime: /usr/lib/jvm/java-8-openjdk-amd64/jre
Default locale: en_US, platform encoding: UTF-8
OS name: "linux", version: "4.14.79+", arch: "amd64", family: "unix"


#### Clone Anserini git repo

In [0]:
!git clone https://github.com/castorini/Anserini.git
!cd Anserini && mvn clean package appassembler:assemble

Cloning into 'Anserini'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 6950 (delta 29), reused 79 (delta 23), pack-reused 6828
Receiving objects: 100% (6950/6950), 11.14 MiB | 11.72 MiB/s, done.
Resolving deltas: 100% (3616/3616), done.
[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.7.0/maven-compiler-plugin-3.7.0.pom (11 kB at 24 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/30/maven-plugins-30.pom (10 kB at 308 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/30/maven-parent-30.pom (41 kB at 984 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/18/apache-18.pom (16 kB at 540 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2

#### Get CAR17 corpus and index data

In [0]:
# !wget http://trec-car.cs.unh.edu/datareleases/v2.0/paragraphCorpus.v2.0.tar.xz
# !tar -xvf paragraphCorpus.v2.0.tar.xz -C /content/Anserini/

In [0]:
# !cd /content/Anserini/ && sh target/appassembler/bin/IndexCollection -collection CarCollection \
# -generator LuceneDocumentGenerator -threads 500000 -input /content/Anserini/paragraphCorpus -index \
# lucene-index.car17a.pos+docvectors+rawdocs -storePositions -storeDocvectors \
# -storeRawDocs

In [0]:
# tar index
# !cd /content/Anserini/ && tar czf lucene-index.car17-large.tar.gz lucene-index.car17a.pos+docvectors+rawdocs

In [0]:
# save index to gs bucket
# !gsutil cp /content/Anserini/lucene-index.car17-large.tar.gz gs://cs378_bert/lucene-index-large.tar.gz

#### Download the Lucene index

In [0]:
!gsutil cp gs://cs378_bert/lucene-index-large.tar.gz /tmp/lucene-index-large.tar.gz
!cd /tmp && tar -xvzf lucene-index-large.tar.gz -C /content/Anserini/

Copying gs://cs378_bert/lucene-index-large.tar.gz...
\
Operation completed over 1 objects/14.1 GiB.                                     
lucene-index.car17a.pos+docvectors+rawdocs/
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.tip
lucene-index.car17a.pos+docvectors+rawdocs/_2.fnm
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.tim
lucene-index.car17a.pos+docvectors+rawdocs/_2.nvd
lucene-index.car17a.pos+docvectors+rawdocs/_2_Lucene50_0.tim
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.doc
lucene-index.car17a.pos+docvectors+rawdocs/_2_Lucene50_0.doc
lucene-index.car17a.pos+docvectors+rawdocs/_3.nvd
lucene-index.car17a.pos+docvectors+rawdocs/_0.fnm
lucene-index.car17a.pos+docvectors+rawdocs/_0_Lucene70_0.dvd
lucene-index.car17a.pos+docvectors+rawdocs/_1.tvx
lucene-index.car17a.pos+docvectors+rawdocs/_0.si
lucene-index.car17a.pos+docvectors+rawdocs/_1.fdx
lucene-index.car17a.pos+docvectors+rawdocs/_3.si
lucene-index.car17a.pos+docvectors+rawdocs/_2.nvm
lucen

#### Install dependencies

In [0]:
!pip3 install pyjnius

!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_squad.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py

--2019-04-19 20:40:04--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.004s  

2019-04-19 20:40:05 (9.99 MB/s) - ‘modeling.py’ saved [37922/37922]

--2019-04-19 20:40:05--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6258 (6.1K) [text/plain]
Saving to: ‘optimizati

### Using Anserini 

In [0]:
# Only run this once
import jnius_config
jnius_config.set_classpath('Anserini/target/anserini-0.4.1-SNAPSHOT-fatjar.jar')

import nltk
nltk.download('punkt')

In [0]:
questions = ['What is Google?', 'Who is Michael Jordan?']

In [0]:
import run_squad
from nltk.tokenize import sent_tokenize
from jnius import autoclass
JString = autoclass('java.lang.String')
JSearcher = autoclass('io.anserini.search.SimpleSearcher')

searcher = JSearcher(JString('Anserini/lucene-index.car17a.pos+docvectors+rawdocs'))

examples = []
for question in questions:
  hits = searcher.search(JString(question),20)
  for hit in hits:
    example = run_squad.SquadExample("0", question, sent_tokenize(hit.content))
    examples.append(example)
    
print(examples)

[qas_id: 0, question_text: What is Google?, doc_tokens: [E. Pariser, author of The Filter Bubble, explains how there are differences that search personalization has on both Facebook and Google. Facebook implements personalization when it comes to the amount of things people share and what pages they "like". An individual's social interactions, whose profile they visit the most, who they message or chat with are all indicators that are used when Facebook uses personalization. Rather than what people share being an indicator of what is filtered out, Google takes into consideration what we "click" to filter out what comes up in our searches. In addition, Facebook searches are not necessarily as private as the Google ones. Facebook draws on the more public self and users share what other people want to see. Even while tagging photographs, Facebook uses personalization and face recognition that will automatically assign a name to face. In terms of Google, users are provided similar websites

In [0]:
#these download all the reqiured BERT material, not sure if they are all required yet
!gsutil cp -r gs://cs378_bert/uncased_L-12_H-768_A-12  /content/BERT/ # copies uncased model
!gsutil cp -r gs://cs378_bert/model_repo  /content/BERT/
!gsutil cp -r gs://cs378_bert/bert-tfhub  /content/BERT/  
!gsutil cp -r gs://cs378_bert/bert-checkpoints  /content/BERT/
 

# BERTsirini - setting up env


Download Required material

In [0]:
#these download all the reqiured BERT material, not sure if they are all required yet
!gsutil cp -r gs://cs378_bert/uncased_L-12_H-768_A-12  /content/BERT/ # copies uncased model
!gsutil cp -r gs://cs378_bert/model_repo  /content/BERT/
!gsutil cp -r gs://cs378_bert/bert-tfhub  /content/BERT/  
!gsutil cp -r gs://cs378_bert/bert-checkpoints  /content/BERT/
 

TPU settings

In [0]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import zipfile
from matplotlib import pyplot as plt
%matplotlib inline
import sys
import datetime
import json
import tensorflow as tf

In [0]:
# Make sure we're connected to a TPU
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please change runtime type'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

In [0]:
#NOT SURE IF WE NEED THIS RIGHT NOW

# BUCKET = 'cs378_bert' #@param {type:"string"}
# assert BUCKET, 'Must specify an existing GCS bucket name'
# REPO ='model_repo'
# BUCKET_OUTPUT_DIR = 'gs://{}/{}/outputs'.format(BUCKET,REPO)
# BUCKET_PREFIX = "gs://cs378_bert"
# tf.gfile.MakeDirs(BUCKET_OUTPUT_DIR)
# print('***** Model output directory: {} *****'.format(BUCKET_OUTPUT_DIR))

#
# !wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/run_squad.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py

In [0]:
import run_classifier
import modeling
import tokenization
import run_squad
from run_squad import SquadExample
import optimization
LEARNING_RATE = 2e-5

# Setup task specific model and TPU running config.
BERT_PRETRAINED_DIR = 'gs://cs378_bert/model_repo/outputs/' 
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')

model_fn = run_classifier.model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  num_labels=len(label_list),
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True
)

# OUTPUT_DIR = OUTPUT_DIR.replace('bert-tfhub', 'bert-checkpoints')
# tf.gfile.MakeDirs(OUTPUT_DIR)

# estimator_from_checkpoints = tf.contrib.tpu.TPUEstimator(
#   use_tpu=True,
#   model_fn=model_fn,
#   config=get_run_config(OUTPUT_DIR),
#   train_batch_size=TRAIN_BATCH_SIZE,
#   eval_batch_size=EVAL_BATCH_SIZE,
#   predict_batch_size=PREDICT_BATCH_SIZE,
# )